# Модели LSTM для прогнозирования временных рядов

**Цели работы:**

0. Научиться подготавливать данные для LSTM. Разаработать метод скользящего окна для каждой модели.
1. Разработать модель LSTM для
    - одномерного 
    - многомерного 
    - многошагового прогнозирования временных рядов
2. Для каждой модели дать оценку предсказанным значениям, используя метрики: 
    - Mean Squared Error
    - Root Mean Squared Error
    - Mean Absolute Error


https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/

In [ ]:
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, RepeatVector, TimeDistributed
from tensorflow import keras
import numpy as np
from numpy import array

## Rolling window

In [ ]:
# univariate data preparation
 
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

## Vanilla LSTM

In [ ]:
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# choose a number of time steps
n_steps = 3

# split into samples
X, y = split_sequence(raw_seq, n_steps)

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

# define model
model = keras.Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(X, y, epochs=200, verbose=0)

# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

## Stacked LSTM

In [ ]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# choose a number of time steps
n_steps = 3

# split into samples
X, y = split_sequence(raw_seq, n_steps)

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

# define model
model = keras.Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# fit model
...

# demonstrate prediction
...

In [ ]:
np.shape(X)

## Bidirectional LSTM

In [ ]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = keras.Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
...

# demonstrate prediction
...


## Многомерные LSTM-модели

#### Ряд с несколькими входами

Мы можем преобразовать эти три массива данных в единый набор данных, где каждая строка — это временной шаг, а каждый столбец — это отдельный временной ряд. Это стандартный способ хранения параллельных временных рядов в файле CSV.

In [ ]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])

out_seq = array([in_seq1[i] + i*np.sin(in_seq2[i]) for i in range(len(in_seq1))])

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))
print(dataset)

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [ ]:
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print('X.shape, y.shape = ', X.shape, y.shape)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

Vanilla LSTM

In [ ]:
n_features = 2

# define model
model = keras.Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(X, y, epochs=200, verbose=0)

# demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

Stacked LSTM

In [ ]:
# define model
model = keras.Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

#### Несколько параллельных временных рядов
Задача альтернативных временных рядов возникает, когда имеется несколько параллельных временных рядов и для каждого из них необходимо спрогнозировать значение.



In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [ ]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print('X.shape, y.shape = ', X.shape, y.shape)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

Stacked LSTM

In [ ]:
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
model = keras.Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(X, y, epochs=400, verbose=0)

# demonstrate prediction
x_input = array([[70,75,145], [80,85,165], [90,95,185]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

### Многошаговые модели LSTM
Задача прогнозирования временных рядов, требующая прогнозирования нескольких временных шагов в будущем, может называться многошаговым прогнозированием временных рядов.

В частности, это задачи, в которых горизонт или интервал прогнозирования превышает один временной шаг.

Существует два основных типа моделей LSTM, которые можно использовать для многошагового прогнозирования:

1. Модель векторного вывода
2. Модель кодера-декодера
Прежде чем рассмотреть эти модели, давайте сначала рассмотрим подготовку данных для многошагового прогнозирования.

In [ ]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

#### Модель векторного вывода
Как и другие типы моделей нейронных сетей, LSTM может напрямую выводить вектор, который можно интерпретировать как многошаговый прогноз.

Этот подход рассматривался в предыдущем разделе, где один временной шаг каждого выходного временного ряда прогнозировался как вектор.

Как и в случае с LSTM для одномерных данных в предыдущем разделе, подготовленные образцы должны быть сначала переформированы. LSTM ожидает, что данные будут иметь трехмерную структуру [ выборки, временные шаги, признаки ], а в этом случае у нас есть только один признак, поэтому переформирование будет простым.

In [ ]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# choose a number of time steps
n_steps_in, n_steps_out = 3, 2

# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

# define model
model = keras.Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=50, verbose=0)

# demonstrate prediction
x_input = array([100, 110, 120])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

#### Модель кодера-декодера
Модель, специально разработанная для прогнозирования выходных последовательностей переменной длины, называется LSTM-кодировщик-декодер .

Модель была разработана для задач прогнозирования, в которых имеются как входные, так и выходные последовательности, так называемые задачи «последовательность-к-последовательности» или seq2seq, например, при переводе текста с одного языка на другой.

Эту модель можно использовать для многошагового прогнозирования временных рядов.

Как следует из названия, модель состоит из двух подмоделей: кодера и декодера.

**Кодер** — это модель, отвечающая за чтение и интерпретацию входной последовательности. Выходные данные кодера — это вектор фиксированной длины, который представляет собой интерпретацию последовательности моделью. Кодер традиционно представляет собой модель Vanilla LSTM, хотя могут использоваться и другие модели кодеров, такие как Stacked, Bidirectional и CNN.

**Декодер** использует выходной сигнал кодера в качестве входного сигнала.

Сначала выходной сигнал кодера фиксированной длины повторяется один раз для каждого требуемого временного шага в выходной последовательности.

In [ ]:
# define model
model = keras.Sequential()

# Кодер
model.add(LSTM(100, activation='relu', input_shape=(n_steps_in, n_features)))

# Декодер
model.add(RepeatVector(n_steps_out))


model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(X, y, epochs=100, verbose=0)
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

## Многомерные многошаговые модели LSTM
В предыдущих разделах мы рассмотрели одномерное, многомерное и многошаговое прогнозирование временных рядов.

Можно смешивать и сопоставлять различные типы моделей LSTM, представленные до сих пор для различных проблем. Это также применимо к проблемам прогнозирования временных рядов, которые включают многомерное и многошаговое прогнозирование, но это может быть немного сложнее.

В этом разделе мы приведем краткие примеры подготовки данных и моделирования для многомерного многошагового прогнозирования временных рядов в качестве шаблона для облегчения этой задачи, а именно:

1. Многоканальный вход, многошаговый выход.
2. Множественный параллельный вход и многошаговый выход.

Пожалуй, самым большим камнем преткновения является подготовка данных, поэтому именно на ней мы сосредоточим свое внимание.

### Многоступенчатый выход с несколькими входами

Существуют такие задачи прогнозирования многомерных временных рядов, когда выходной ряд отделен, но зависит от входного временного ряда, и для выходного ряда требуется несколько временных шагов.

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out-1
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [ ]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))

# choose a number of time steps
n_steps_in, n_steps_out = 3, 2

# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
print("X.shape, y.shape = ", X.shape, y.shape)
    
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

Векторный выход Stacked LSTM.

In [ ]:
# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]

# define model
model = keras.Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(X, y, epochs=200, verbose=0)

# demonstrate prediction
x_input = array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

### Множественный параллельный вход и многошаговый выход
Проблема с параллельными временными рядами может потребовать прогнозирования нескольких временных шагов каждого временного ряда.

Мы можем использовать последние три временных шага из каждого из трех временных рядов в качестве входных данных для модели и прогнозировать следующие временные шаги каждого из трех временных рядов в качестве выходных данных.

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [ ]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))

# choose a number of time steps
n_steps_in, n_steps_out = 3, 2

# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
print("X.shape, y.shape = ", X.shape, y.shape)

# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


Encoder-Decoder

In [ ]:
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
model = keras.Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=300, verbose=0)
# demonstrate prediction
x_input = array([[60, 65, 125], [70, 75, 145], [80, 85, 165]])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

**Реализуйте модель "Векторного вывода"**
